In [1]:
import os 

#Insert API key
os.environ['OPENAI_API_KEY'] = '***'

In [2]:
import sys
sys.path.append("../")

#Import your prompt - we have a system message, starting prompt and iteration prompt
# ADSC is hard search with all operators (A), 
# and all tools (D (data), C (context), S (scratchpad))

from prompt_ADSC import SYS_MSG, IGNITE, ITER


In [3]:
#Import experiment class
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

#experiment file all different experiment settings in Python class
from experiment import allTools, noContext, noScratchpad, noData

In [9]:
#Import data
import pandas as pd
#We will try Langmuir
df = pd.read_csv('datasets/Langmuir1918nitrogen.csv')
data = df.to_dict(orient='list')

all_keys = list(data.keys())
all_values = list(data.values())
y = all_values[1]
x1 = all_values[0]

dep_var = str(y)
indep_var = [str(x1)]


context= "The data is about nitrogen adsorbing onto mica where the independent variable (x1) is pressure, and the dependent variable (y) is loading."


In [10]:
import json
#No. of expressions generated in each LLM call
N = 3
temp = 0.7
model = "gpt-4-0613"

#Increase iterations here
total_iterations = 5 

sys_msg= SYS_MSG
ignite_msg = IGNITE
iter_msg = ITER

results_fromAllruns = []
iteration_infos_fromAllruns = []


# Run the experiment n times
for i in range(3):
    print(f"Running experiment {i+1}...")

    # Create a new instance of the experiment class with your settings
    #options = allTools, noContext, noData, noScratchpad
    experiment = allTools(dep_var, indep_var, N, temp, context, sys_msg, ignite_msg, iter_msg, model)
    # Run the experiment and store the results
    results, all_expressions, iteration_info, usage_list, total_chain_run_time, all_expressionsLLM  = experiment.run(total_iterations)
    
    results_fromAllruns.append({
        'CombResults': results, 
        'all_expressions': all_expressions, 
        'usage_list': usage_list, 
        'total_chain_run_time': total_chain_run_time,
        'LLM raw expressions': all_expressionsLLM
    })
    
    # Store the iteration info
    iteration_infos_fromAllruns.append(iteration_info)

    print(f"Experiment {i+1} finished. Cost: {experiment.cost()}")

    directory_path = "tests/HS_allTools/Langmuir"
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)  # This will create all intermediate-level directories needed to contain the leaf directory

    with open(f"{directory_path}/PromptsUsed.txt", 'w') as f:
        f.write(f"##Sytstem Message## {SYS_MSG}\n")
        f.write(f"##Starting Prompt## {IGNITE}\n")
        f.write(f"##Iteration Prompt## {ITER}\n")

    with open(f"{directory_path}/run{i+1}.txt", 'w') as f:
        f.write("Final Results:\n")
        for res in results:
            json.dump(res, f, indent=2)
            f.write("\n")
        f.write(f"Cost for this run: ${experiment.cost()}\n")
        f.write(f"Total chain run time: {total_chain_run_time} seconds.\n")
        f.write("\n\nIteration Information:\n")
        for info in iteration_info:
            json.dump(info, f, indent=2)
            f.write("\n")

Running experiment 1...
Iteration:Seed
SciPy feedback used for this iteration:
None
LLM thoughts:
Scratch Pad:

Firstly, It can be observed that the relationship between the dependent and independent variables is non-linear. The dependent variable, y, decreases as the independent variable, x1, decreases. 

A suitable equation could be a power or exponential function or a combination of both. 

Given that we are dealing with physical quantities, it's reasonable to first try out a power law function of the form y = c0*x1^c1 where c0 and c1 are constants. 

For the second expression, we could try an exponential function of the form y = c0*exp(-c1*x1) since the y values decrease as x1 decreases. It's also common in physics for exponential decay processes to be described by such equations.

Lastly, we could also try a function of the form y = c0*sqrt(x1) + c1 since the relationship between y and x1 could be inversely proportional to the square root of x1.
New equations generated:
['c[0]*x1*